In [1]:
from sogym.env import sogym
train_env = sogym(mode='train',observation_type='box_dense')

In [ ]:
from sogym.expert_generation import generate_dataset
generate_dataset(num_threads =2, num_samples=10,dataset_folder = "/Users/thomasrochefort/Documents/GitHub/so_gym/dataset/topologies")

In [3]:
from imitation.algorithms import bc
from imitation.algorithms.adversarial.airl import AIRL
from imitation.rewards.reward_nets import BasicShapedRewardNet
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, SubprocVecEnv
from imitation.util import logger as imit_logger

# Create the vectorized environment
num_cpu = 4 # Number of processes to use
venv= make_vec_env(lambda:train_env, n_envs=num_cpu,vec_env_cls=SubprocVecEnv)

policy_kwargs = dict(
    net_arch=dict(pi=[32,32,32], vf=[32,32,32]),
)
learner = PPO("MlpPolicy",
    env=venv,
    n_steps=2048//num_cpu,  
    batch_size=2048, 
    tensorboard_log="tb_logs",

)

custom_logger = imit_logger.configure(
        folder="test_logs",
        format_strs=["tensorboard", "stdout"],
    )

In [ ]:
rng = np.random.default_rng(0)

bc_trainer = bc.BC(
    observation_space=train_env.observation_space,
    action_space=train_env.action_space,
    demonstrations=trajectories,
    rng=rng,
    custom_logger =custom_logger,
    ent_weight=3e-3,
    l2_weight=1e-3,
    batch_size=1024,
    policy = learner.policy
)


In [ ]:
reward_net = BasicShapedRewardNet(
    venv.observation_space, venv.action_space,
    #reward_hid_sizes=(32,32),
    #potential_hid_sizes=(32, 32),
    #discount_factor=1.0,
    #dropout_prob=0.5
)

#learner=PPO.load('imit',env=venv,device='cuda')

airl_trainer = AIRL(
    demonstrations=trajectories,
    demo_batch_size=2048,
    debug_use_ground_truth=True,
    init_tensorboard=True,
    log_dir="test_logs",
    n_disc_updates_per_round=1,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
    custom_logger= custom_logger

)

In [ ]:
airl_trainer.train(4000000)  # Note: set to 300000 for better results

In [ ]:
learner.save('imit',)

In [ ]:
env=sogym(mode='train',observation_type='box_dense')
env = make_vec_env(lambda:env, n_envs=1,vec_env_cls=SubprocVecEnv)
model=learner.load('imit')
#model.set_env(env)

In [ ]:
obs=env.reset()
dones=False

while dones== False:
    action, _states = model.predict(obs,deterministic=False)
    print(action)
    obs, rewards, dones, info = env.step(action)
  
print("Reward:",rewards[0])
fig=env.env_method('plot')[0]
fig